In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from PIL import Image
from io import BytesIO
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class PropertyScraper:
    def __init__(self, url):
        self.driver = webdriver.Chrome()
        self.driver.get(url)
        self.all_data = pd.DataFrame()

    def set_search_parameters(self, sro, local, year):
        self.select_element_by_id("ctl00_ContentPlaceHolder1_ddl_sro_s", sro)
        self.select_element_by_id("ctl00_ContentPlaceHolder1_ddl_loc_s", local)
        self.select_element_by_id("ctl00_ContentPlaceHolder1_ddl_year_s", year)

    def select_element_by_id(self, element_id, value):
        element = self.driver.find_element(By.ID, element_id)
        element.send_keys(value)

    def get_captcha_image(self):
        captcha_img_element = self.driver.find_element(By.XPATH, "//div[@class='btn btn-sm']//img")
        captcha_img_url = captcha_img_element.get_attribute("src")
        image = Image.open(BytesIO(urlopen(captcha_img_url).read()))
        image.show()

    def submit_captcha(self, captcha_text):
        self.driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_txtcaptcha_s").send_keys(captcha_text)
        self.driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_btn_search_s").click()

    def scrape_current_page(self):
        soup = BeautifulSoup(self.driver.page_source, "html.parser")
        table = soup.find("table")
        df = pd.read_html(str(table))[0]
        df = df.dropna(subset=['First Party'])
        df = df.iloc[:-1]
        df.drop(["Reg.No", "Deed Type"], axis=1, inplace=True)
        df.rename(columns={"Area": "Area (sq. feet)"}, inplace=True)
        df["Area (sq. feet)"] = df["Area (sq. feet)"].str.replace("Sq. Feet", "").str.strip()
        df.reset_index(drop=True, inplace=True)
        return df

    def wait_for_element(self, by, value, timeout=10):
        try:
            element_present = EC.presence_of_element_located((by, value))
            WebDriverWait(self.driver, timeout).until(element_present)
        except TimeoutException:
            return None
        return self.driver.find_element(by, value)

    def click_next_page(self):
        next_page = self.wait_for_element(By.ID, "ctl00_ContentPlaceHolder1_gv_search_ctl13_Button2")
        if next_page and next_page.is_enabled() and next_page.is_displayed():
            self.driver.execute_script("arguments[0].click();", next_page)
            return True
        return False

    def scrape_all_data(self):
        self.all_data = self.scrape_current_page()
        while True:
            try:
                time.sleep(2)
                if self.click_next_page():
                    time.sleep(2)
                    new_data = self.scrape_current_page()
                    self.all_data = pd.concat([self.all_data, new_data], ignore_index=True)
                else:
                    break
            except NoSuchElementException:
                break
        self.all_data = self.all_data.reset_index(drop=True)

    def close_driver(self):
        self.driver.quit()

def main():
    url = "https://esearch.delhigovt.nic.in/Complete_search.aspx"
    sro = "Central -Asaf Ali (SR III)"
    local = "Ajmal Khan Road"
    year = "2021-2022"

    # Initialize the scraper
    scraper = PropertyScraper(url)
    
    # Set search parameters
    scraper.set_search_parameters(sro, local, year)

    # Show captcha image
    scraper.get_captcha_image()

    # Get captcha input from the user
    captcha_text = input("Please enter the captcha text: ")

    # Submit the captcha
    scraper.submit_captcha(captcha_text)

    # Scrape all data
    scraper.scrape_all_data()

    # Save the scraped data to a Excel file
    scraper.all_data.to_excel("property_data.xlsx", index=False)

    # Close the WebDriver
    scraper.close_driver()

if __name__ == "__main__":
    main()
